# Create Heterostructure Example (Simplified)

This notebook demonstrates how to create a heterostructure with any number of materials using the **new simplified `create_heterostructure` function**.

This approach replaces the complex multi-step interface creation with a single function call that automatically handles strain matching and layer stacking.

<h2 style="color:green">Usage</h2>

1. **Set up the notebook and install packages**
2. **Configure structure parameters for any number of materials**
3. **Import materials from Standata or user uploads**
4. **Create heterostructure with a single function call**

## Summary

1. **Prepare the Environment:** Set up the notebook and install packages
2. **Configure Materials:** Define parameters for each layer in the stack
3. **Create Stack:** Use simplified `create_heterostructure()` function
4. **Visualize:** Preview the resulting heterostructure
5. **Pass to Runtime:** Pass the final heterostructure to the external runtime

## Key Improvements:
- **Simplified Configuration**: Replace complex multi-step setup with simple parameter lists
- **Single Function Call**: Use `create_heterostructure()` instead of manual interface creation
- **Automatic Optimization**: Built-in strain matching and supercell optimization
- **Flexible Stacking**: Supports any number of materials in the stack

## Notes

1. For more information, see [Introduction](Introduction.ipynb)

<!-- # TODO: use a hashtag-based anchor link to interface creation documentation above -->


## 1. Configuration Parameters

Configure the heterostructure parameters including materials, layer thicknesses, and gaps.


In [ ]:
from types import SimpleNamespace

# Structure parameters for each layer - add or remove layers as needed
STRUCTURE_PARAMS = [
    SimpleNamespace(
        # First material (substrate)
        name="Silicon",  # Material name from Standata or upload
        slab_params=SimpleNamespace(
            miller_indices=(0, 0, 1),
            thickness=3,  # atomic layers
            xy_supercell_matrix=[[1, 0], [0, 1]],
        ),
        termination_formula=None,  # Use default termination
    ),
    SimpleNamespace(
        # Second material
        name="SiO2",  # Material name from Standata or upload  
        slab_params=SimpleNamespace(
            miller_indices=(1, 1, 1),
            thickness=3,  # atomic layers
        ),
        termination_formula=None,  # Use default termination
    ),
    SimpleNamespace(
        # Third material
        name="HfO2.*MCL",  # Material name from Standata or upload
        slab_params=SimpleNamespace(
            miller_indices=(0, 0, 1), 
            thickness=1,  # atomic layers
        ),
        termination_formula=None,  # Use default termination
    ),
    # Add more materials here as needed...
]

# Gap distances between layers (in Angstroms)
# Length should be one less than number of materials
GAPS = [3.0, 3.0]  # Gaps between materials

# Final vacuum above the heterostructure (in Angstroms)
VACUUM = 20.0


## 2. Install Packages

The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).


In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install("mat3ra-api-examples", deps=False)
    await micropip.install('mat3ra-utils')
    from mat3ra.utils.jupyterlite.packages import install_packages

    await install_packages("specific_examples")


## 3. Get Input Materials

Materials are loaded from Standata using the names specified in `STRUCTURE_PARAMS` or from user uploads via `get_materials()`.


In [ ]:
from mat3ra.standata.materials import Materials
from mat3ra.made.tools.build import MaterialWithBuildMetadata
from utils.jupyterlite import get_materials

# Try to get materials from Standata first, then fallback to uploads
crystals = []
uploaded_materials = get_materials(globals()) if get_materials else []

for i, structure_param in enumerate(STRUCTURE_PARAMS):
    try:
        # Try to get from Standata
        standata_material = Materials.get_by_name_first_match(structure_param.name)
        crystal = MaterialWithBuildMetadata.create(standata_material)
        print(f"Found '{structure_param.name}' in Standata")
    except:
        # Fallback to uploaded materials
        if i < len(uploaded_materials):
            crystal = MaterialWithBuildMetadata.create(uploaded_materials[i])
            print(f"Using uploaded material {i} for '{structure_param.name}'")
        else:
            # Use first uploaded material as fallback
            crystal = MaterialWithBuildMetadata.create(uploaded_materials[0] if uploaded_materials else Materials.get_by_name_first_match("Silicon"))
            print(f"Using fallback material for '{structure_param.name}'")
    
    crystals.append(crystal)


## 4. Preview Original Materials

Visualize the loaded materials before creating the heterostructure.


In [ ]:
from utils.visualize import visualize_materials as visualize

# Create materials with titles for visualization
materials_with_titles = [
    {"material": crystal, "title": f"Layer {i}: {STRUCTURE_PARAMS[i].name}"} 
    for i, crystal in enumerate(crystals)
]

visualize(materials_with_titles, repetitions=[3, 3, 1], rotation="0x")


## 5. Create the Heterostructure
### 5.1. Create Stack Component Dictionaries

Create a list of dictionaries for each layer's parameters using the simplified approach.


In [ ]:
from mat3ra.made.tools.helpers import StackComponentDict

stack_component_dicts = [
    StackComponentDict(
        crystal=crystals[i],
        miller_indices=structure_param.slab_params.miller_indices,
        thickness=structure_param.slab_params.thickness,
        xy_supercell_matrix=structure_param.slab_params.xy_supercell_matrix if hasattr(structure_param.slab_params, 'xy_supercell_matrix') else None,
        termination_top_formula=structure_param.termination_formula,
    ) for i, structure_param in enumerate(STRUCTURE_PARAMS)
]

print(f"Created {len(stack_component_dicts)} stack components:")
for i, component in enumerate(stack_component_dicts):
    print(f"  Layer {i}: {STRUCTURE_PARAMS[i].name} - Miller: {component.miller_indices}, Thickness: {component.thickness}")


### 5.2. Create the Heterostructure

Using the `create_heterostructure` function with automatic strain matching and optimization.


In [ ]:
from mat3ra.made.tools.helpers import create_heterostructure

heterostructure = create_heterostructure(
    stack_component_dicts=stack_component_dicts,
    gaps=GAPS,
    vacuum=VACUUM,
    use_conventional_cell=True,
    optimize_layer_supercells=True,
)

print(f"Successfully created heterostructure with {len(crystals)} materials")
print(f"Final structure has {heterostructure.basis.number_of_atoms} atoms")


## 6. Visualize the Heterostructure


In [ ]:
visualize(
    heterostructure,
    repetitions=[1, 1, 1],
    viewer="wave",
    rotation='0x'
)

## 7. Save the Heterostructure



In [ ]:
from utils.jupyterlite import set_materials

# Set heterostructure name based on all materials
material_names = [param.name for param in STRUCTURE_PARAMS]
# heterostructure.name = " - ".join(material_names) + " Heterostructure"

set_materials(heterostructure)